In [1]:
import numpy as np
import scipy.sparse as sp
import torch

In [2]:
node_class = torch.load(f'./dataset/IMDB/imdb_node_class.pt')  # (4780, 3)
community= torch.load(f'./dataset/IMDB/imdb_community_label.pt') 

In [3]:
node_class_val = torch.sum(node_class, dim=1)


In [4]:
non_zero_indices = torch.nonzero(node_class_val).squeeze()

In [5]:
non_zero_indices

tensor([   0,    1,    2,  ..., 4775, 4777, 4778])

In [6]:
# 在范围 [0, 3287] 内随机生成 200 个数
random_indices = torch.randint(0, 3287, (600,))


In [7]:
query_node= non_zero_indices[random_indices]

In [9]:
query_node.shape

torch.Size([600])

In [10]:
torch.save(query_node, 'query_node_600.pt')

In [39]:
dense_community=    community.to_dense()

In [43]:
train_query_community = dense_community[query_node[:200]]
value_query_community = dense_community[query_node[200:400]]
test_query_community = dense_community[query_node[400:600]]

In [47]:
torch.save(train_query_community, 'train_query_and_community.pt')
torch.save(value_query_community, 'value_query_and_community.pt')
torch.save(test_query_community, 'test_query_and_community.pt')

In [49]:

torch.save(query_node[:200], 'train_node_id.pt')
torch.save(query_node[200:400], 'value_node_id.pt')
torch.save(query_node[400:600], 'test_node_id.pt')

In [97]:
import scipy.io as sio
data = sio.loadmat('./dataset/IMDB/imdb5k.mat')

In [2]:
imdb_MAM_adj = data['MAM']
imdb_MDM_adj = data['MDM']

NameError: name 'data' is not defined

In [57]:
import numpy as np
import scipy.sparse as sp
import torch

In [99]:
imdb_node_feature=data['feature']

In [100]:
imdb_node_label = data['label']

In [101]:
imdb_node_label


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 0.]])

In [60]:
# 将普通的NumPy数组转换为稀疏张量
def array_to_sparse_tensor(array):
    # 获取非零元素的索引和数据
    row, col = np.nonzero(array)
    data = array[row, col]

    # 将索引和数据转换为PyTorch张量
    indices = torch.tensor([row, col], dtype=torch.int)
    values = torch.tensor(data, dtype=torch.int)

    # 创建PyTorch稀疏张量
    sparse_tensor = torch.sparse_coo_tensor(indices, values, array.shape)
    return sparse_tensor


In [61]:
imdb_MAM_adj_tensor = array_to_sparse_tensor(imdb_MAM_adj)
imdb_MDM_adj_tensor = array_to_sparse_tensor(imdb_MDM_adj)

In [62]:
imdb_node_label_tensor = torch.tensor(imdb_node_label, dtype=torch.float)
imdb_node_feature_tensor = torch.tensor(imdb_node_feature, dtype=torch.float)


In [63]:
imdb_node_feature_tensor=imdb_node_feature_tensor.to_sparse()

In [68]:
# 将两个密集张量合并成一个形状为[2, 100, 100]的密集张量
merged_dense = torch.stack([imdb_MAM_adj_tensor, imdb_MDM_adj_tensor], dim=0)

In [ ]:
merged_dense = torch.stack([imdb_MAM_adj_tensor, imdb_MDM_adj_tensor], dim=0)

In [70]:
merged_feature = torch.stack([imdb_node_feature_tensor, imdb_node_feature_tensor], dim=0)

In [71]:
merged_feature

tensor(indices=tensor([[   0,    0,    0,  ...,    1,    1,    1],
                       [   0,    0,    0,  ..., 4779, 4779, 4779],
                       [ 431,  646,  712,  ..., 1105, 1167, 1216]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(2, 4780, 1232), nnz=46960, layout=torch.sparse_coo)

In [72]:
imdb_node_feature_tensor

tensor(indices=tensor([[   0,    0,    0,  ..., 4779, 4779, 4779],
                       [ 431,  646,  712,  ..., 1105, 1167, 1216]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(4780, 1232), nnz=23480, layout=torch.sparse_coo)

In [77]:
torch.save(merged_feature, 'imdb_features.pt')
torch.save(merged_dense, 'imdb_adjs.pt')

In [54]:
torch.save(imdb_MAM_adj_tensor, 'imdb_MAM_adj.pt')
torch.save(imdb_MDM_adj_tensor, 'imdb_MDM_adj.pt')
torch.save(imdb_node_feature_tensor, 'imdb_node_feature_tensor.pt')
torch.save(imdb_node_label_tensor, 'imdb_node_label_tensor.pt')

In [46]:
imdb_MDM_adj_tensor

tensor(indices=tensor([[   0,    0,    0,  ..., 4779, 4779, 4779],
                       [   0,   25,  282,  ..., 4076, 4508, 4779]]),
       values=tensor([1, 1, 1,  ..., 1, 1, 1]),
       size=(4780, 4780), nnz=21018, dtype=torch.int32, layout=torch.sparse_coo)

In [22]:
dense_tensor = torch.tensor(imdb_MAM_adj, dtype=torch.float)


In [23]:
torch.save(dense_tensor, 'tensor.pt')

In [8]:
# 将NumPy数组转换为PyTorch稀疏张量
def array_to_torch_sparse(array):
    row, col = np.nonzero(array)
    data = array[row, col]
    indices = torch.tensor([row, col], dtype=torch.long)
    values = torch.tensor(data, dtype=torch.float)
    return torch.sparse_coo_tensor(indices, values, array.shape)


In [9]:

imdb_MAM_adj = array_to_torch_sparse(imdb_MAM_adj)

/var/folders/wz/rkdpw75151934rc10njt33940000gn/T/ipykernel_2399/874544538.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  indices = torch.tensor([row, col], dtype=torch.long)


In [10]:
imdb_MAM_adj

tensor(indices=tensor([[   0,    0,    0,  ..., 4778, 4778, 4779],
                       [   0,  300,  447,  ..., 3202, 4778, 4779]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(4780, 4780), nnz=98010, layout=torch.sparse_coo)

In [11]:
torch.save(imdb_MAM_adj, 'sparse_tensor.pt')

In [12]:
loaded_dense_tensor = torch.load('sparse_tensor.pt')

In [13]:
loaded_dense_tensor

tensor(indices=tensor([[   0,    0,    0,  ..., 4778, 4778, 4779],
                       [   0,  300,  447,  ..., 3202, 4778, 4779]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(4780, 4780), nnz=98010, layout=torch.sparse_coo)

In [ ]:
sparse_matrix = sp.csr_matrix(imdb_MAM_adj)

# 将稀疏矩阵保存为.npz文件
sp.save_npz('imdb_MAM_adj.npz', sparse_matrix)

In [24]:
sparse_matrix_MDM = sp.csr_matrix(imdb_MDM_adj)

# 将稀疏矩阵保存为.npz文件
sp.save_npz('imdb_MDM_adj.npz', sparse_matrix_MDM)

In [25]:
sparse_matrix_F = sp.csr_matrix(imdb_node_feature)

# 将稀疏矩阵保存为.npz文件
sp.save_npz('imdb_node_feature.npz', sparse_matrix_F)

In [31]:
np.save('imdb_node_label.npy', imdb_node_label)

In [78]:
adjs = torch.load(f'./dataset/IMDB/imdb_adjs.pt') #(2, 4780, 4780) 稀疏tensor
features = torch.load(f'./dataset/IMDB/imdb_node_features.pt') #(2, 4780, 1232) 稀疏tensor
node_class = torch.load(f'./dataset/IMDB/imdb_node_class.pt') #(4780, 3)

In [82]:
# 将 adjs 张量的两个矩阵相加
adj_sum = adjs[0] + adjs[1]
adj_sum=adj_sum.to_dense()
# 将结果中大于1的值设置为1
adj_sum = torch.where(adj_sum > 1, torch.tensor(1), adj_sum)


In [89]:
adj_sum


tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 1, 0, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 1]], dtype=torch.int32)

In [90]:
node_class

tensor([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        ...,
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 0.]])

In [91]:
node_class.shape

torch.Size([4780, 3])

In [96]:
import torch
import networkx as nx

# 假设 adj_sum 是形状为 [4780, 4780] 的邻接矩阵
# 假设 node_class 是形状为 [4780, 3] 的节点类别矩阵

# 将邻接矩阵转换为 NetworkX 图
G = nx.from_numpy_array(adj_sum.numpy())

# 初始化社区矩阵
community_matrix = torch.zeros((4780, 4780))

# 遍历每个节点
for node in range(4780):
    # 捕获5跳邻居，得到一个五跳之内的子图
    subgraph_nodes = nx.single_source_shortest_path_length(G, node, cutoff=3).keys()
    subgraph = G.subgraph(subgraph_nodes)
    
    # 判断子图内部的节点是否都属于同一类别
    subgraph_classes = node_class[list(subgraph.nodes)]
    if torch.all(subgraph_classes.sum(dim=0) <= 1):
        # 如果子图内部的节点都属于同一类别，则将其添加到社区矩阵中
        for neighbor in subgraph.nodes:
            community_matrix[node, neighbor] = 1


KeyboardInterrupt: 

In [93]:
adj_sum.numpy()

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int32)